In [1]:
#import necessary libraries
import os
from tqdm import tqdm
from joblib import Parallel, delayed
from multiprocessing import Process

In [2]:
#input hashes list to store all the hashes
input_hashes = []

#input hashes list to store all the hashes
output_hashes = []

In [3]:
def get_lists(input_type, month):
    
    #hashes list to store all the hashes
    hashes = []
    
    #read files
    if input_type == "inputs":
        filename = "./Data/edges2010/inputs2010_{}.txt".format(str(month+1))
    if input_type == "outputs":
        filename = "./Data/edges2010/outputs2010_{}.txt".format(str(month+1))

    with open(filename) as file:
        for line in file:
            #split by tab char to get the details present
            details = line.split("\t")

            #hash present at the second value
            #append in the list
            hashes.append(details[1])
            
    return hashes

In [4]:
#as there is data file for each month
month = 12

#calling the above function parallely for inputs
input_hash = Parallel(n_jobs=-1)(delayed(get_lists)("inputs", i) for i in tqdm(range(month)))

100%|██████████████████████████████████████████| 12/12 [00:00<00:00, 130.14it/s]


In [5]:
for hashes in input_hash:
    input_hashes.extend(hashes)
        
print("Total unique input hashes are: {}".format(len(input_hashes)))

Total unique input hashes are: 117483


In [6]:
#calling the above function parallely for outputs
output_hash = Parallel(n_jobs=-1)(delayed(get_lists)("outputs", i) for i in tqdm(range(month)))

100%|█████████████████████████████████████████| 12/12 [00:00<00:00, 6285.17it/s]


In [7]:
for hashes in output_hash:
    output_hashes.extend(hashes)
        
print("Total unique output hashes are: {}".format(len(output_hashes)))

Total unique output hashes are: 117483


In [8]:
len(list(set(input_hashes) & set(output_hashes)))

117483

In [9]:
#create mapping
mapping = dict(zip(set(output_hashes), range(len(output_hashes))))

In [10]:
#create a updated folder to store the updated text files
folder_name = "./Data/edges2010/updated"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

In [11]:
def update_files(input_type, month):
    
    #read files
    if input_type == "inputs":
        filename = "./Data/edges2010/inputs2010_{}.txt".format(str(month+1))
        updated_filename = "./Data/edges2010/updated/inputs2010_{}.txt".format(str(month+1))

    if input_type == "outputs":
        filename = "./Data/edges2010/outputs2010_{}.txt".format(str(month+1))
        updated_filename = "./Data/edges2010/updated/outputs2010_{}.txt".format(str(month+1))

    file = open(filename, 'r')
    contents = file.read()
    
    for key, value in mapping.items():
        contents = contents.replace(key, str(value))
    
    updated_file = open(updated_filename, 'w')
    updated_file.write(contents)
    
    file.close()
    updated_file.close()

In [12]:
#updated inputs
jobs = []
for i in tqdm(range(12)):
    p = Process(target=update_files, args=("inputs",i,))
    jobs.append(p)
    p.start() 
#Parallel(n_jobs=-1)(delayed(update_files)("inputs", i) for i in tqdm(range(month)))

100%|██████████████████████████████████████████| 12/12 [00:00<00:00, 164.30it/s]


In [13]:
#updated outputs
jobs = []
for i in tqdm(range(12)):
    p = Process(target=update_files, args=("outputs",i,))
    jobs.append(p)
    p.start() 
#Parallel(n_jobs=-1)(delayed(update_files)("inputs", i) for i in tqdm(range(month)))

100%|███████████████████████████████████████████| 12/12 [00:00<00:00, 69.47it/s]
